## This notebook demonstrates the creation of a Sentinel-1 or Sentinel-2 composite only or a composite that combines both Sentinel-1 and Sentinel-2
> #### Input: Reference points (GEE FeatureCollection) containing the classes of interest as a property.
> #### Arguments: A start and end data, cloud threshold, area of interest, image composite interval
> #### Output: Invasive Tree species map

**Step 1**: Install and load modules  
**Step 2**: Prepare time series composite and additional covariates  
**Step 3**: Prepare dataset for model training and evaluation  
**Step 4**: Export data (optional)

## Step 1: Install and load modules

In [2]:
%pip install watermark geemap geeml -q
!git clone https://github.com/Geethen/Invasive_Species_Mapping.git
import sys
sys.path.insert(0,'/content/Invasive_Species_Mapping/code')

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lightgbmlss 0.1.0 requires torch~=1.10.0, but you have torch 1.12.0 which is incompatible.

[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
fatal: destination path 'Invasive_Species_Mapping' already exists and is not an empty directory.


In [1]:
%load_ext watermark

In [2]:
import ee
# ee.Authenticate()
ee.Initialize()
import geemap

# Add module to environment varibles
import sys
MODULE_FULL_PATH = r'C:\Users\coach\myfiles\postdoc\code\Invasive_Species_Mapping\code'
sys.path.insert(1, MODULE_FULL_PATH)

# Load python modules with preprocessing functions. 
from timeSeriesFunctions import prepareTS
from covariateFunctions import prepareCovariates
from trainDataFunctions import prepareTrainingData
from modelFitFunctions import prepareModel
from metricFunctions import prepareMetrics
from geeml.utils import eeprint

In [3]:
%watermark -v -m --iversions

Python implementation: CPython
Python version       : 3.9.13
IPython version      : 8.4.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 2, GenuineIntel
CPU cores   : 12
Architecture: 64bit

geemap: 0.20.0
sys   : 3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:50:36) [MSC v.1929 64 bit (AMD64)]
ee    : 0.2



In [16]:
%pip install geemap -q --upgrade

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Step 2: Prepare time series composite

### In this example we specifiy the start and dates to create a composite for 2017 (The Date_end is exclusive and therfore the reason it is for 1 Jan 2018). 


In [4]:
# start and end dates for time series
Date_Start = ee.Date('2017-01-01')
Date_End = ee.Date('2018-01-01')


Next, a 30-day interval is specified. This will result a monthly cloud-free Sentinel-2 image.

In [5]:
# how many days to summarise in each image e.g 30 days = 12 images per year
day_int = 30 #step size


#### Next, a 30% probability is specified i.e., a pixel with a probability of >= 30% of containing cloud cover will be masked based on the s2cloudless cloud probability dataset in Google Earth Engine.


In [6]:

# cloud probabillity threshold
CLOUD_THRESH=30

### Next, the bands to include in the final composite are selected. Note the ndvi, ndwi, ndre, nbr and evi bands are computed by default.

In [7]:
# which bands to keep
BANDSS2 = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6','B7', 'B8', 'B8A', 'B9', 'B11','B12', 'ndvi','ndwi','ndre','nbr','evi']
BANDSS1 = ['VV', 'VH', 'VH/VV']

#### Lastly, the Area of Interest (AOI) is defined. This determines the extent for which the monthly composite for 2017 is computed.


In [8]:
# define your study area
study_area1 = ee.Geometry.Polygon([[[30.555572509765607, -24.407014032416797],
          [31.065750122070295, -24.40888986482876],
          [31.065750122070295, -24.40888986482876],
          [31.049957275390607, -25.121539326337704],
          [30.560379028320295, -25.133972597740186],
          [30.555572509765607, -24.407014032416797]]], None, False)

With these parameters configured we can go on to create three types of composites

1. A Sentinel-2 time series composite
2. A Sentinel-1 time series composite
3. A combined Sentinel-2 and Sentinel-1 composite

### 1. A Sentinel-2 composite

Returns a 12-month time-series for each of the bands specified (BANDSS2) using the L1C data (if s2_level =2, a level L2A composite can be created). Cloud masked pixels are gap filled by using pixels from a 6-month median image.

In [10]:

s2TS = prepareTS(Date_Start,Date_End,day_int,study_area1,CLOUD_THRESH,BANDSS2).timeSeries(satellite = 'S2', s2_level = 1)
s2TS

### 2. A Sentinel-1 composite

Returns a 12-month time-series for each of the bands specified (BANDSS1).

In [12]:
s1TS = prepareTS(Date_Start,Date_End,day_int,study_area1,CLOUD_THRESH,BANDSS1).timeSeries(satellite = 'S1')

### 3. A combined Sentinel-1 and Sentinel-2 composite

No initialisation parameters are specified, since the composites have already been prepared

In [22]:
s1s2TS = prepareTS().joinS1S2(s1TS, s2TS)
s1s2TS

EEException: Join.apply, argument 'secondary': Invalid type.
Expected type: FeatureCollection.
Actual type: Image<[band0_VV, band0_VH, band0_VH/VV, band1_VV, band1_VH, band1_VH/VV, band2_VV, band2_VH, band2_VH/VV, band3_VV, band3_VH, band3_VH/VV, band4_VV, band4_VH, band4_VH/VV, band5_VV, band5_VH, band5_VH/VV, band6_VV, band6_VH, band6_VH/VV, band7_VV, band7_VH, band7_VH/VV, band8_VV, band8_VH, band8_VH/VV, band9_VV, band9_VH, band9_VH/VV, band10_VV, band10_VH, band10_VH/VV, band11_VV, band11_VH, band11_VH/VV, band12_VV, band12_VH, band12_VH/VV]>.